<a href="https://colab.research.google.com/github/Steven-Macas/PIA/blob/Python/StevenMacas_Scrapy_PIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Crear una araña que recorra una página web y extraigas los datos***

---



In [ ]:
#version3.steven
import scrapy
#from proyecto_imdb.items import ProyectoImdbItem


class ImdbSpider(scrapy.Spider):
    _page = 1
    name = 'imdb'
    allowed_domains = ['www.imdb.com']
    initial_url = 'https://www.imdb.com'
    start_urls = ['https://www.imdb.com/search/title/?genres=comedy&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=3396781f-d87f-4fac-8694-c56ce6f490fe&pf_rd_r=W15HBXJWV2E2BNKRC8EV&pf_rd_s=center-1&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr1_i_1']

    #Método generadora
    def parse(self, response):
      #Dos métodos: CSS() y xpath()
      #item = ProyectoImdbItem()

      movies = response.css('div.lister > div.lister-list > div.lister-item')
      for movie in movies:
          self.position = movie.css('span.lister-item-index::text').get()#div.lister-item-content > h3.lister-item-header > span.lister-item-index
          self.title = movie.css("h3.lister-item-header > a::text").get()
          self.date = movie.css("span.lister-item-year::text").get()
          href = movie.css("a::attr(href)").get()
          link = self.initial_url + href
          stars = movie.css("p > a::text").getall()

          #La otra forma que se intentó

          """position = movie.css('span.lister-item-index::text').get()#div.lister-item-content > h3.lister-item-header > span.lister-item-index
          title = movie.css("h3.lister-item-header > a::text").get()
          date = movie.css("span.lister-item-year::text").get()
          stars = movie.css('h3.lister-item-header > p::text').get()
          item = {
            'position':position,
            'title':title,
            'date':date,
            'stars':stars,
            'resennas_users':None,
            'resennas_criticos':None
          }"""

          #Meterse en cada pelicula para conseguir las reseñas de usuarios y criticos
          #link_movie = self.initial_url + movie.css('h3 > a::attr(href)').get()
          #yield scrapy.Request(url = link_movie, callback = self.parse_detail, cb_kwargs={'imdb_item':item} )
          #Como hubo problemas, al final no se conseguió que funcionara con éxito del todo, en este caso, al final no se usará la funcion parseMovie pero se mostrará como ejemplo de como se hubiera llevado a cabo

          yield{
            #'author': authors,
            'position':self.position,
            'title': self.title,
            'date': self.date,
            'stars': stars,
            'link':link
          }


      
      # go to next page
      next_page = response.css("div.desc a.lister-page-next::attr(href)").get()
      if self._page < 15:
          yield response.follow(next_page, callback=self.parse)
          self._page+=1


    def parseMovie(self,response,item):

      rese_usu1 = response.css('li:nth-child(1) > a > span > span')
      rese_usu = rese_usu1.css('span::text').get()
      rese_criti2 = response.css('li:nth-child(2) > a > span > span')
      rese_criti = rese_criti2.css('span::text').get()

      item['resennas_users'] = rese_usu
      item['resennas_users'] = rese_criti

      #Los datos obtenidos lo devolvemos
      yield{
            'position':item['position'],
            'title': item['title'],
            'date': item['date'],
            'stars': item['stars'],
            'resennas_users':item['resennas_users'],
            'resennas_criticos':item['resennas_criticos']
          }